In [1]:
# Using PuLP dependency to seriously simplify setting up LP problem. It's just a wrapper to help set things up,
#  but it does come with a built-in solver (which we will also use)
from pulp import LpVariable, LpProblem, LpMinimize, lpDot, LpStatus, value
import numpy as np

from src.graph import Graph
from src.graph._preProcessing import connectGraph, removeBackEdges
from src.data.basicTypes import Ingredient, IngredientCollection, Recipe
from factory_graph import ProgramContext

In [2]:
class LPSolver:
    def __init__(self, graph):
        self.graph = graph
        self.variables = []
        # self.variable_idx_counter = 0 # Autogen current "head" index for variable number
        # self.system = []
        self.solved_vars = None # Result from linear solver

        self.lookup = {} # (machine, product, direction, multi_idx) -> variable index
        self.edge_from_perspective_to_index = {} # (edge, machine_id) -> variable index

def graphPreProcessing(self):
    connectGraph(self)
    # if not self.graph_config.get('KEEP_BACK_EDGES', False):
    #     removeBackEdges(self)
    Graph.createAdjacencyList(self)

def linearProgrammingSolver(self: ProgramContext, project_name: str, recipes: list[Recipe], graph_config: dict):
    g = Graph(project_name, recipes, self, graph_config=graph_config)
    self._graph = g # For test access
    graphPreProcessing(g)
    print(g.adj)
    
    


In [3]:
context = ProgramContext()

context.graph_gen = linearProgrammingSolver # Override solver
context.generate_one("power/fish/methane.yaml")

defaultdict(<function createAdjacencyList.<locals>.<lambda> at 0x0000018A9EEBE7A0>, {'source': defaultdict(<class 'list'>, {'O': [('source', '0', 'pams fish')]}), '0': defaultdict(<class 'list'>, {'I': [('source', '0', 'pams fish')], 'O': [('0', '1', 'methane')]}), '1': defaultdict(<class 'list'>, {'I': [('0', '1', 'methane')], 'O': [('1', 'sink', 'biogas')]}), 'sink': defaultdict(<class 'list'>, {'I': [('1', 'sink', 'biogas')]})})


In [4]:
# This is me figuring out edge cases.

"""
- m: centrifuge
  tier: LV
  I:
    pams fish: 1
  O:
    methane: 96
  eut: 5
  dur: 19
  number: 1
  cost_priority:
    pams fish: 1
  # target:
  #   # methane: 250
  #   pams fish: 1
- m: distillery
  tier: LV
  I:
    methane: 125
  O:
    biogas: 375
  eut: 7
  dur: 1
"""

#  

problem = LpProblem("test", LpMinimize)
cost_inputs = ["pams fish"]
recipe_vars = ["pams fish", "methane"]
items = ["pams fish", "methane", "biogas"]
target_vector = [0,0,1000]
ing_matrix = {
  "fuge": [-1, 96, 0],
  "dist": [0, -125, 375]
}
# How do you figure out which variables should be inputs?
input_vectors = {
  "fish_in": [1, 0, 0],
}
# rec_mat = np.array(list(recipe_vectors.values())+ list(input_vectors.values()))
# display(rec_mat)
# rec_mat.transpose()
all_vectors = ing_matrix.copy()
all_vectors.update(input_vectors)
# Item constraints are transpose of values of constraint vectors (all vectors)
item_constraints = np.array(list(all_vectors.values())).transpose()


In [5]:
# problem.variables()
# Create variables
recipe_vars = LpVariable.dicts("rec", all_vectors.keys(), lowBound=0, cat="Continuous")

# Add objective
problem += recipe_vars["fish_in"]
# Add constraints
for item_constraint_vec, target in zip(item_constraints, target_vector):
    problem += lpDot(recipe_vars.values(), item_constraint_vec) >= target
# problem += lpDot(x.values(), target_vector) >= 0

problem.solve()
LpStatus[problem.status]

'Optimal'

In [6]:
for v in problem.variables():
    print(v.name, "=", v.varValue)

rec_dist = 2.6666667
rec_fish_in = 3.4722222
rec_fuge = 3.4722222


In [7]:
# That works, now let's mess around with ways to identify input candidates
# Cases that need to be inputs are useful to report the user, 
# but we want to find "eliminated" variables as described by https://github.com/ClaudeMetz/FactoryPlanner/blob/master/modfiles/backend/calculation/matrix_engine.lua
# - that is, variables that are net-zero after all loops/etc and don't need to be system inputs.

# I'm theorizing that the "eliminated" variables might be findable via the reduced row echelon form of the matrix.

from sympy import Matrix
# m = Matrix(np.hstack([item_constraints, np.array(target_vector).reshape(-1,1)]))
# m.rref()

# One example is a loop in the palladium component of the platline,
# which inputs palladium enriched ammonia and loops that and pall met pow dust
# [ammonia, pall enriched ammonia, pall met pow, pall salt, repre pall]
# and cols are:
# [lcr pall met pow recycle, lcr repre, sifter salt recycle]
pall_loop = [
    [-1000, 0,      0],
    [1000,  -9000,  0],
    [-1,    -9,     0.95],
    [0,     16,     -1],
    [0,     2,      0],
]
 
positive_cost_cycle = Matrix(pall_loop)
 
positive_cost_cycle.rref()
#  Oh yeah, duh, rref on an over-defined matrix. Uhhhh
# This matrix converts recipe executions to items produced, 
# and solving it finds necessary recipes executions/unit time to produce desired items
# We're looking for input-item-to-output item conversions. How do we find those?
# - 

(Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0]]),
 (0, 1, 2))

In [8]:
positive_cost_cycle.singular_value_decomposition()

(Matrix([
 [  -0.0123427693881819,     0.999920219926959, -0.00266637715277894],
 [    0.999921784104044,     0.012337578452739, -0.00197762613328934],
 [ 0.000975236276396803,   0.00201218158793192,    0.688745946762978],
 [ -0.00175569605886587,  -0.00179957317684196,   -0.724995172566176],
 [-0.000219462003270191, -0.000224946177417711,   1.0320007302374e-6]]),
 Matrix([
 [9056.0851669197,                0,                0],
 [              0, 993.812582692507,                0],
 [              0,                0, 1.37930382200068]]),
 Matrix([
 [  0.111777170775024,   -0.99373329625207,  3.6777441977233e-6],
 [ -0.993733296258832,  -0.111777170773151, 7.11721275761968e-7],
 [2.96173288126787e-7, 3.73425105498543e-6,   0.999999999992984]]))

In [9]:
positive_cost_cycle * positive_cost_cycle.transpose()

Matrix([
[ 1000000, -1000000,    1000,       0,      0],
[-1000000, 82000000,   80000, -144000, -18000],
[    1000,    80000, 82.9025, -144.95,    -18],
[       0,  -144000, -144.95,     257,     32],
[       0,   -18000,     -18,      32,      4]])

In [10]:
simpler_pos_cost = Matrix([
    [-1, 2],
    [0.25, -1]
])
simpler_pos_cost.rref()
# I don't think this is going to work for determining which should be inputs, but I think I can figure it out using LP

(Matrix([
 [1, 0],
 [0, 1]]),
 (0, 1))

In [11]:
# One example is a loop in the palladium component of the platline,
# which inputs palladium enriched ammonia and loops that and pall met pow dust
# [ammonia, pall enriched ammonia, pall met pow, pall salt, repre pall]
# and cols are:
# [lcr pall met pow recycle, lcr repre, sifter salt recycle]
pall_loop = [
    [-1000, 0,      0],
    [1000,  -9000,  0],
    [-1,    -9,     0.95],
    [0,     16,     -1],
    [0,     2,      0],
]
target_vector = [0, 0, 0, 0, 10]

# New theory: for anything that is ever an input, make an input vector, but tell LP to minimize number used
# Possible upgrade: seek to maximize earliness of inputs in a topological sort or derivative.

In [12]:
# Priorities: [recipe tax, pall enriched ammonia, additional vector cost]
priority_ratio = 9000/0.95 # Smallest/biggest

from pulp import LpProblem, LpMinimize, LpBinary, LpVariable, value, lpSum, lpDot

problem = LpProblem("min input test", LpMinimize)
recipe_vars = LpVariable.dicts("recipe",  ["lcr pall met pow recycle", "lcr repre", "sifter salt recycle"],0)
variables =  ["ammonia", "pall enriched ammonia", "pall met pow", "pall salt", "repre salt"] 
# Filtered only by things that are ever an input
inputs = ["ammonia", "pall enriched ammonia", "pall met pow", "pall salt"] 
explicit_inputs = ["pall enriched ammonia"]
explicit_input_amounts = LpVariable.dicts("input", explicit_inputs, 0)
# Filtered to remove explicit inputs
additional_inputs = ["ammonia", "pall met pow", "pall salt"] 
additional_input_subtractors = LpVariable.dicts("input", additional_inputs, 0)
additional_input_switches = LpVariable.dicts("in_switch", additional_inputs, cat=LpBinary)
try:
    for item, recipe_coeffs, target in zip(variables, pall_loop, target_vector):
        # To reach each target amount, use a combination of the recipe outputs and switched inputs
        # The amount for each input is unlimited, but there is a high cost for switching each one on
        if item in explicit_inputs:
            input_term = explicit_input_amounts[item]
        elif item in additional_inputs:
            input_term = additional_input_subtractors[item] * additional_input_switches[item]
        else:
            input_term = 0
        problem += lpDot(recipe_vars.values(), recipe_coeffs) + input_term >= target
        
    # Objective, in increasing order of priority:
    # 1: Per-recipe tax
    # (2, n-1): explicit costs to minimize (explicit inputs)
    # n: High-cost additional inputs
    problem += (priority_ratio**0 * lpSum(recipe_vars.values())
        + priority_ratio**1 * explicit_input_amounts["pall enriched ammonia"]
        + priority_ratio**2 * lpSum(additional_input_switches))
except TypeError as e:
    print("Well poop.")
    print(e)

Well poop.
Non-constant expressions cannot be multiplied


c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [13]:
# Little essay about a new approach to solve this which-inputs-should-we-choose problem:

# Alright, so making the linear optimizer solve for minimal inputs while also solving the problem ain't going to work.
# ... because this formulation is quadratic and I can't think of a nice model that isn't.
# ... we could get rid of the toggle and treat new inputs cumulatively as the highest priority class
# ... but that has lots of edge cases and naturally gives a higher weight to minimizing large-number stuff, like fluids.
# ... we could counteract *that* by normalizing quantities (so 9000L of oxygen becomes 9kL of oxygen, or a less nice unit)
# ... but I can't think of a good normalization scheme that doesn't have problems with the different ranges of quantities between recipes.

# New insight: Focus on solving for minimal inputs first, then solve problem.
# Model this as a cover problem: we want to find the smallest set of inputs which reaches the inputs of all recipes.

# We can once again use Linear Optimization to solve this problem.
# We want to find the minimum number of inputs we must provide so that every recipe *can* be run.
# We want to prioritize hitting every recipe, *then* minimizing the number of inputs.
# Note that the linear optimizer might not use all resources if it choses not to include a relevant recipe in a solution.
# Note that if you can run a parent recipe (e.g. ammonia and oxygen for nitric acid), 
# ... you can run descendent recipes (e.g. nitric acid [and ammonia] for ammonium chloride)

# I'm going to model this as a bipartite graph in my head.
# - Every resource that is ever an input to any recipe becomes a requirement (constraint in LP terms)
#       - (one for oxygen, hydrochloric, ilmenite, whatever)
#       - These are nodes on the right-hand side of the graph.
#       - These are the inputs to "cover" with as few new/raw/actual inputs as possible. 
#       - I will call them "requirement" resources to avoid confusing myself.
# - We also take this same list of inputs and make a binary (on or off) variable for every resource
#       - Picture these as nodes on the left-hand side of the graph
#       - Turning on these inputs corresponds to selecting that variable as an input.
#       - I will call them "providable" resources to avoid confusing myself.
# - In graph terms, we put an edge between every providable resource and the requirements it covers. This includes:
#       - The resource itself (providing oxygen obviously covers oxygen requirements)
#       - For every recipe which uses this resource, cover all of its outputs
#       - For all of *those* resources, keep covering - this becomes DFS.
#       - In LP terms, every constraint is sum(every binary variable that covers this required resource) >= 1
# - Finally, the objective is the minimize the sum of all those binary variables (aka find fewest selected inputs)
# - As a preprocessing step, all requirements that are indirectly covered by explicitly provided inputs can be deleted.

# Something in my head tickles that this is NP-Complete, but I don't feel like putting in the work to test/prove it.
# Regardless, LP solvers can totally tackle NPC problems like Knapsack (and they're reasonably optimized about it)

# One big issue with this approach: selecting a valuable resource before a loop 
# (for example, palladium salt dust, which loops around and indirectly supplies *many* recipes)
# ... will have high value and cover many resources. 
# Theory #1: We could use a heuristic-based "cyclic graph toposort" to minimize back-edges (loop-backs)
# ... use that sort to form a DAG that mostly moves ingredient->output, and then use this algorithm
# Theory #2: By the nature of most GTNH recipe graphs, especially when at least one start-of-the-chain ingredient has been provided,
# ... a solution which uses just-before-a-loop resources as an input won't usually be "minimal".
# Further, this algorithm doesn't need to *perfectly* solve this problem, 
# ... because the user can fix wierd behavior by explicitly providing more resources.


In [14]:
# One example is a loop in the palladium component of the platline,
# which inputs palladium enriched ammonia and loops that and pall met pow dust
# [ammonia, pall enriched ammonia, pall met pow, pall salt, repre pall]
# and cols are:
# [lcr pall met pow recycle, lcr repre, sifter salt recycle]
pall_loop = [
    [-1000, 0,      0],
    [1000,  -9000,  0],
    [-1,    -9,     0.95],
    [0,     16,     -1],
    [0,     2,      0],
]
target_vector = [0, 0, 0, 0, 10]
item_recipe_covers = {
    "ammonia": {"lcr pall met pow recycle": {"pall enriched ammonia"}},
    "pall met pow": {"lcr pall met pow recycle": {"pall enriched ammonia"}},
    "pall enriched ammonia": {"lcr repre": {"pall salt", "repre pall"}},
    "pall met pow": {"lcr repre": {"pall salt", "repre pall"}},
    "pall salt":{"sifter salt recycle": {"pall met pow"}}
}
recipes = {
    "lcr pall met pow recycle": {
        "I": {"ammonia":1000, "pall met pow": 1},
        "O": {"pall enriched ammonia": 1000},
    },
    "lcr repre": {
        "I": {"pall enriched ammonia": 9000, "pall met pow": 9},
        "O": {"pall salt": 16, "repre pall": 2},
    },
    "sifter salt recycle": {
        "I": {"pall salt": 1},
        "O": {"pall met pow": 0.95},
    }
}

In [15]:
import re
from collections import Counter
from src.data.loadMachines import recipesFromConfig
def genSlug(s, try_acronym=False):
    slug = re.sub(r"[^a-zA-Z0-9_]", "", re.sub(r"\W+", "_", s.lower()))
    words = slug.split("_")
    if len(slug) > 25 or (try_acronym and len(words)>=2):
        return "".join(word[0] for word in words)
    else:
        return slug

def genRecipeNames(recipes):
    counter = Counter()
    # name_map = {"sink": "sink", "source": "source"}
    names = []
    for recipe in recipes:
        name = genSlug(recipe.machine, True) + "_" + genSlug(recipe.I[0].name)
        if name in counter:
            counter[name] += 1
            name += f"_{counter[name]}"
        names.append(name)
    return names

In [16]:
def stripBrackets(ing):
    prefix = False
    if ing[:2] == '\u2588 ':
        prefix = True
    stripped = ing.split(']')[-1].strip()
    if prefix and stripped[:2] != '\u2588 ': 
        stripped = '\u2588 ' + stripped
    return stripped

def getItemCovers(recipes):
    item_covers = {}

    item_to_covered_recipes = {}
    recipe_to_covered_items = {}
    items_to_cover = set()
    # Just use indices to identify recipes here
    for covered_recipe, io in enumerate(recipes):
        for ing in io.I:
            ing_name = stripBrackets(ing.name)
            item_to_covered_recipes[ing_name] = item_to_covered_recipes.get(ing_name, []) + [covered_recipe]
            items_to_cover.add(ing_name)
        for out in io.O:
            out_name = stripBrackets(out.name)
            recipe_to_covered_items[covered_recipe] = recipe_to_covered_items.get(covered_recipe, []) + [out_name]


    # Cheeky DFS for each item to figure out what it covers.
    # This could be done all-at-once with some shenanigans, but I think we want to not allow loops on a per-item basis
    # My gut is telling me that will be a slightly more restricted/less-prone-to-error heuristic.
    for item in items_to_cover:
        used_recipes = set()
        covered_items = {item}
        frontier = [item]
        while not len(frontier) == 0:
            covered_item = frontier.pop()
            if covered_item not in item_to_covered_recipes: continue # item covers no recipes
            for covered_recipe in item_to_covered_recipes[covered_item]:
                if covered_recipe in used_recipes: continue # Recipe already used (we've looped)
                used_recipes.add(covered_recipe)
                if covered_recipe not in recipe_to_covered_items: continue # recipe covers no items (we deleted its items in preprocessing)
                for outgoing_item in recipe_to_covered_items[covered_recipe]:
                    if outgoing_item in covered_items: continue # We've already hit this item
                    covered_items.add(outgoing_item)
                    frontier.append(outgoing_item)
        item_covers[item] = covered_items
    return item_covers
recipes = recipesFromConfig("devtest/palladium_loop.yaml")
getItemCovers(recipes)
# Only ammonia covers ammonia (good), but I think this loopy example is a bad example.

{'palladium enriched ammonia': {'palladium enriched ammonia',
  'palladium metallic powder dust',
  'palladium salt dust',
  'reprecipitated palladium dust'},
 'palladium salt dust': {'palladium enriched ammonia',
  'palladium metallic powder dust',
  'palladium salt dust',
  'reprecipitated palladium dust'},
 'palladium metallic powder dust': {'palladium enriched ammonia',
  'palladium metallic powder dust',
  'palladium salt dust',
  'reprecipitated palladium dust'},
 'ammonia': {'ammonia',
  'palladium enriched ammonia',
  'palladium metallic powder dust',
  'palladium salt dust',
  'reprecipitated palladium dust'}}

In [17]:
bauxite_recipes = recipesFromConfig("223_bauxite_line.yaml")
baux_covers = getItemCovers(bauxite_recipes)

display({item: len(covered) for (item, covered) in baux_covers.items()})
set(baux_covers.keys()) - set(baux_covers["bauxite dust"])
# That spooked me at first but turns out yeah, bauxite line has one input: bauxite. Lets try another.

{'titanium tetrachloride': 11,
 'magnesium dust': 11,
 'bauxite dust': 15,
 'carbon dust': 11,
 'chlorine': 11,
 'rutile dust': 12,
 'magnesium chloride dust': 11,
 'sodium dust': 11,
 'salt': 11,
 'hot titanium ingot': 2,
 'carbon monoxide': 11}

set()

In [18]:
bauxite_recipes = recipesFromConfig("2200_tungstate.yaml")
baux_covers = getItemCovers(bauxite_recipes)

display({item: len(covered) for (item, covered) in baux_covers.items()})
set(baux_covers.keys()) - set(baux_covers["endstone dust"])
# Okay, this finally proves a concern I had - that this binary "cover/not cover" approach
# won't handle when an input would reach an input, but not *enough*
# for tungstate, endstone covers most inputs, but byproduct hydrogen only covers *some* of the needed amount
# I thought this might be a non-event, but nay, it's an event. We'll need to start calculating ratios.

{'endstone dust': 17,
 'tungstate dust': 13,
 'tungstic acid dust': 3,
 'hydrogen': 2,
 'hydrochloric acid': 13,
 'sodium hydroxide dust': 12,
 'calcium chloride dust': 12,
 'water': 13,
 'scheelite dust': 12,
 'sodium dust': 12,
 'salt': 12,
 'tungsten trioxide dust': 2,
 'sodium tungstate': 12}

{'hydrochloric acid', 'water'}

In [19]:
# New essay on how to avoid doing what the last essay was talking about 
# (solving for min inputs one at a time is hard, lets do it all at once again!)

# It might be possible to frame this as an LP (actually just an rref),
# but I think it would still require a DFS from me to figure out which recipes to include in the solve
# so I'm going to run the ratios myself.
# Main Idea: if you run across a recipe you've already seen and think you can cover
#   an additional recipe-ingredient using a byproduct, confirm you have enough of that byproduct to meet that requirement.
# Problem: what if you attempt this when you don't have enough byproduct, but later another recipe gives you more of that byproduct?
# Problem: what if you dfs and hit this recipe with the byproduct ingredient first and the main ingredient later?
#   - In this case, you show as producing excess of the main ingredient

# Example of this: "insufficient_byproduct_fake.png", where a sodium tungstate input can either:
#   - Cover the EBF with scheelite, then produce byproduct hydrogen, which is not enough to cover the EBF's hydrogen input 
#       - This requires an extra hydrogen input, which is desired behavior.
#   - Cover the salt LCR with salt for hydrogen, then cover the EBF with hydrogen. 
#       - When we reach the EBF with scheelite, it looks like we have excess - great! (not desired behavior)

#.... poop. I can't think of a way to prioritize "main ingredients" like scheelite without requiring a priori knowledge of ingredient value.
#   - An LP problem which asks to maximize reachable ingredients will of course use the route that covers both hydrogen and scheelite
#   - An LP problem which minimizes reachable ingredients... finds no reachable ingredients?
#   - What about an LP problem that is constrained to reach as many ingredients as possible but minimizes out these byproduct events?

# AH! Theory: a reformulation of the quadratic issue before. We find a quantity of each input that is  
# "The most this problem could possibly use" - get to that later. Point is, that's wired to a binary input we want to minimize.
# Then, another variable subtracts from that binary-with-big-coefficient number.
# If the binary is off, that subtractor goes to zero (no less, because of constraints/bounds), 
# .... but if it isn't, it's equal to difference from big-number.
# Big-number minus subtractor yields amount of the switched-on input. Could work!

# But how do we figure out the most an input could possibly need? I don't love the "1 trillion or so ought to do" solution.
# I think the priority ratio math will still apply: sum of targets * (ratio of smallest to biggest value in problem)
# But there could be a setup like:
# Biggest-number input (1000 A) produces smallest-number output (0.1 B)
# Second-biggest-number input (999 B) produces second-smallest-number output (0.2 C)
# ... and so on. Those two would require (999/0.2*1000/0.1) = 49950000.0 units of A for 1 C

# This would require some sort of DFS ratio calculation that would be no fun.
# I say we use the "1 trillion or so ought to do strategy" and include a warning if any subtractor is zero (input 100% used)
# ... but no, if that much of an input really is used (maybe a VERY large liquid air distilling problem),
# ... then no solution would be found at all! Wait, not true - another input could be used as well.
# ... We'll go with this solution.

# Choose a BIG_NUMBER that doesn't intrude too far into double precision when LP starts doing its math.
# Doubles have ~15 decimal digits of precision. Let's leave... uh... 6? for recipes, leaving 9 for BIG_NUMBER.
# 1,000,000,000 - A trillion or so ought to do. That wasn't even intentional.
# This problem should be somewhat helped (in not-contrived examples) by input scale normalization (see above).

In [20]:
BIG_NUMBER = 1E15
print("Two digits with 1E15", BIG_NUMBER-2.16, BIG_NUMBER-2.12, BIG_NUMBER-2.09)
# NO, BAD - not enough precision.
BIG_NUMBER = 1E9
print("Eight Digits with 1E9", BIG_NUMBER-2.12345678, BIG_NUMBER-2.12345679, "\nSeven Digits", BIG_NUMBER-2.1234567, BIG_NUMBER-2.1234568)
# I suppose it's not so surprising when math does what it is supposed to do, but it's neat to see that (almost) work.
# Bc floats use exponents, the first 2 gets divided to 1, which is implicit in a float (so that digit doesn't count towards precision).
# I would have wanted to see Seven Digits not work bc it should be 6, but I probably chose numbers that fit
# into the gaps of "Approximately" 6 digits of precision.
# Let's try this approach with some problems and see what happens when we reduce BIG_NUMBER.

Two digits with 1E15 999999999999997.9 999999999999997.9 999999999999997.9
Eight Digits with 1E9 999999997.8765432 999999997.8765432 
Seven Digits 999999997.8765433 999999997.8765432


In [21]:
# New LP Plan:
# Constraints: (recipe contributions) + ingredient_switch*BIG_NUMBER - ingredient_subtractor >= target
# Objective Priorities (low->high):
# 0: Recipe Tax (unweighted sum of recipe amounts to avoid unnecessary work)
# (1, n-2): Explicit Ingredient Priorities (ratio**k * (sum of n-priority explicit ingredient vectors))
# n-1: Additional Ingredient Quantities (sum)
# n: Ingredient Switches (sum)

# Note that this prioritizes not using an ingredient at all over using less of it
# and it prioritizes using explicit ingredients before additional ones. 

In [22]:
# Priorities: [recipe tax, pall enriched ammonia, additional input amounts, additional input switches]
priority_ratio = 9000/0.95 # Smallest/biggest
BIG_NUMBER = 1E9

from pulp import LpProblem, LpMinimize, LpBinary, LpVariable, value, lpSum, lpDot

def solveProblem(big_number):
    problem = LpProblem("min input v2", LpMinimize)
    recipe_vars = LpVariable.dicts("recipe",  ["lcr pall met pow recycle", "lcr repre", "sifter salt recycle"],0)
    variables =  ["ammonia", "pall enriched ammonia", "pall met pow", "pall salt", "repre salt"] 
    # Filtered only by things that are ever an input
    inputs = ["ammonia", "pall enriched ammonia", "pall met pow", "pall salt"] 
    explicit_inputs = ["pall enriched ammonia"]
    explicit_input_amounts = LpVariable.dicts("input", explicit_inputs, 0)
    # Filtered to remove explicit inputs
    additional_inputs = ["ammonia", "pall met pow", "pall salt"]
    additional_input_subtractors = LpVariable.dicts("input", additional_inputs, 0, 1)
    additional_input_switches = LpVariable.dicts("in_switch", additional_inputs, 0, 1, cat=LpBinary)


    for item, recipe_coeffs, target in zip(variables, pall_loop, target_vector):
        # To reach each target amount, use a combination of the recipe outputs and switched inputs
        # The amount for each input is unlimited, but there is a high cost for switching each one on
        if item in explicit_inputs:
            input_term = explicit_input_amounts[item]
        elif item in additional_inputs:
            # input_term = 0
            input_term = big_number * (additional_input_switches[item] - additional_input_subtractors[item])
            # Also restrict input to be strictly positive (no inputting negative items)
            problem += (additional_input_switches[item] - additional_input_subtractors[item]) >= 0 
        else:
            input_term = 0
        problem += lpDot(recipe_vars.values(), recipe_coeffs) + input_term == target

    problem += (priority_ratio**0 * lpSum(recipe_vars.values())
        + priority_ratio**1 * lpSum(explicit_input_amounts)
        + priority_ratio**2 * -1 * lpSum(additional_input_subtractors.values()) # invert (to max)
        + priority_ratio**3 * lpSum(additional_input_switches)
    )
    problem.solve()
    
    print("Problem Status", LpStatus[problem.status])

    def show_values(var_dict):
        for var in var_dict:
            print(var, value(var_dict[var]))
            
    print("\nSwitches")
    show_values(additional_input_switches)
    print("\nSubtractors")
    show_values(additional_input_subtractors)
    print("\nRecipe vars")
    show_values(recipe_vars)
    print("\nExplicit Ingredient Vars")
    show_values(explicit_input_amounts)

    print("\nConstraints: ")
    for item, recipe_coeffs, target in zip(variables, pall_loop, target_vector):
        # To reach each target amount, use a combination of the recipe outputs and switched inputs
        # The amount for each input is unlimited, but there is a high cost for switching each one on
        if item in explicit_inputs:
            input_term = value(explicit_input_amounts[item])
        elif item in additional_inputs:
            input_term = big_number * (value(additional_input_switches[item]) - value(additional_input_subtractors[item]))
        else:
            input_term = 0
        recipe_sum = sum(value(v) * coeff for ((name, v), coeff) in zip(recipe_vars.items(), recipe_coeffs))
        coeff_string = " + ".join([f"{value(v)} * {coeff} [{name}]" for ((name, v), coeff) in zip(recipe_vars.items(), recipe_coeffs)])
        print(item, f"{coeff_string} + {input_term} = {recipe_sum+input_term} >= {target}")
solveProblem(1E4)
# That is a correct solution! I think there are some edge cases regarding byproduct handling...
# There are some cases where you *need* a byproduct (no solution otherwise),
# And there are some cases where you *want* a byproduct (a side-recipe cracks e.g. radioactive waste into uranium, etc)
# I'll explore solving that next. 

Problem Status Infeasible

Switches
ammonia 3.1
pall met pow 0.0
pall salt 0.0

Subtractors
ammonia 0.0
pall met pow 0.0
pall salt 0.0

Recipe vars
lcr pall met pow recycle 31.0
lcr repre 5.0
sifter salt recycle 80.0

Explicit Ingredient Vars
pall enriched ammonia 14000.0

Constraints: 
ammonia 31.0 * -1000 [lcr pall met pow recycle] + 5.0 * 0 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 31000.0 = 0.0 >= 0
pall enriched ammonia 31.0 * 1000 [lcr pall met pow recycle] + 5.0 * -9000 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 14000.0 = 0.0 >= 0
pall met pow 31.0 * -1 [lcr pall met pow recycle] + 5.0 * -9 [lcr repre] + 80.0 * 0.95 [sifter salt recycle] + 0.0 = 0.0 >= 0
pall salt 31.0 * 0 [lcr pall met pow recycle] + 5.0 * 16 [lcr repre] + 80.0 * -1 [sifter salt recycle] + 0.0 = 0.0 >= 0
repre salt 31.0 * 0 [lcr pall met pow recycle] + 5.0 * 2 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 0 = 10.0 >= 10


In [23]:
# Find behavior with a nearly-too-small BIG_NUMBER
solveProblem(1E5)

Problem Status Optimal

Switches
ammonia 1.0
pall met pow 0.0
pall salt 0.0

Subtractors
ammonia 0.69
pall met pow 0.0
pall salt 0.0

Recipe vars
lcr pall met pow recycle 31.0
lcr repre 5.0
sifter salt recycle 80.0

Explicit Ingredient Vars
pall enriched ammonia 14000.0

Constraints: 
ammonia 31.0 * -1000 [lcr pall met pow recycle] + 5.0 * 0 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 31000.000000000004 = 3.637978807091713e-12 >= 0
pall enriched ammonia 31.0 * 1000 [lcr pall met pow recycle] + 5.0 * -9000 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 14000.0 = 0.0 >= 0
pall met pow 31.0 * -1 [lcr pall met pow recycle] + 5.0 * -9 [lcr repre] + 80.0 * 0.95 [sifter salt recycle] + 0.0 = 0.0 >= 0
pall salt 31.0 * 0 [lcr pall met pow recycle] + 5.0 * 16 [lcr repre] + 80.0 * -1 [sifter salt recycle] + 0.0 = 0.0 >= 0
repre salt 31.0 * 0 [lcr pall met pow recycle] + 5.0 * 2 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 0 = 10.0 >= 10


In [24]:
# Find behavior with a too-small BIG_NUMBER
solveProblem(1E4)
# This failed how we would want to it - straight-out
# - but I suspect it will stop failing like this as we ease off on byproduct restrictions for byproduct-y problems.

Problem Status Infeasible

Switches
ammonia 3.1
pall met pow 0.0
pall salt 0.0

Subtractors
ammonia 0.0
pall met pow 0.0
pall salt 0.0

Recipe vars
lcr pall met pow recycle 31.0
lcr repre 5.0
sifter salt recycle 80.0

Explicit Ingredient Vars
pall enriched ammonia 14000.0

Constraints: 
ammonia 31.0 * -1000 [lcr pall met pow recycle] + 5.0 * 0 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 31000.0 = 0.0 >= 0
pall enriched ammonia 31.0 * 1000 [lcr pall met pow recycle] + 5.0 * -9000 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 14000.0 = 0.0 >= 0
pall met pow 31.0 * -1 [lcr pall met pow recycle] + 5.0 * -9 [lcr repre] + 80.0 * 0.95 [sifter salt recycle] + 0.0 = 0.0 >= 0
pall salt 31.0 * 0 [lcr pall met pow recycle] + 5.0 * 16 [lcr repre] + 80.0 * -1 [sifter salt recycle] + 0.0 = 0.0 >= 0
repre salt 31.0 * 0 [lcr pall met pow recycle] + 5.0 * 2 [lcr repre] + 80.0 * 0 [sifter salt recycle] + 0 = 10.0 >= 10


In [69]:
# Modify the solver to use generic problems and allow penalized byproducts.

# Priorities: [recipe tax, pall enriched ammonia, additional input amounts, additional input switches, byproducts]
priority_ratio = 500 # hardcoded for now
BIG_NUMBER = 1E3

from pulp import LpProblem, LpMinimize, LpBinary, LpVariable, value, lpSum, lpDot
from collections import Counter

def genSlug(s, try_acronym=False):
    slug = re.sub(r"[^a-zA-Z0-9_]", "", re.sub(r"\W+", "_", s.lower()))
    words = slug.split("_")
    if len(slug) > 40 or (try_acronym and len(words)>=2):
        return "".join(word[0] for word in words if len(word) > 0)
    else:
        return slug[:25]

def genRecipeNames(recipes):
    counter = Counter()
    names = []
    for recipe in recipes:
        represent_ing = recipe.I[0].name if len(recipe.I) > 0 else "nothing"
        name = genSlug(recipe.machine, True) + "_" + genSlug(represent_ing)
        counter[name] += 1
        if counter[name] > 1:
            name += f"_{counter[name]}"
        names.append(name)
    return names

class LpProject:
    def __init__(self, inputs, explicit_inputs, outputs, targets, variables, recipe_names, ing_matrix, target_vector):
        self.inputs = inputs
        self.explicit_inputs = explicit_inputs
        self.outputs = outputs
        self.targets = targets
        self.variables = variables
        self.recipe_names = recipe_names
        self.ing_matrix = ing_matrix
        self.target_vector = target_vector
        
    @staticmethod
    def fromRecipes(recipes):
        inputs = set()
        explicit_inputs = set()
        outputs = set()
        targets = {}
        # Just use indices to identify recipes here
        for recipe, io in enumerate(recipes):
            for ing in io.I:
                ing_name = stripBrackets(ing.name)
                inputs.add(ing_name)
            for out in io.O:
                out_name = stripBrackets(out.name)
                outputs.add(out_name)
            if hasattr(io, "cost"):
                for explicit_input in getattr(io, "cost"):
                    explicit_inputs.add(explicit_input)
            if hasattr(io, "target"):
                for target, quant in getattr(io, "target").items():
                    targets[target] = quant
                    
        if not all(target in outputs for target in targets): 
            raise RuntimeError("Encountered target which is never an output (likely a spelling mistake). targets: " +str(targets))
        if not all(cost in inputs for cost in explicit_inputs):
            raise RuntimeError("Encountered cost/explicit input which is never an input (likely a spelling mistake). costs: " +str(explicit_inputs))
        
        variables = list(inputs | outputs)
        
        recipe_vectors = []
        variable_indices = {var: i for i, var in enumerate(variables)}
        print(variable_indices)
        for recipe in recipes:
            vector = [0] * len(variables)
            for ing in recipe.I:
                vector[variable_indices[stripBrackets(ing.name)]] = -1 * ing.quant
            for out in recipe.O:
                vector[variable_indices[stripBrackets(out.name)]] = out.quant
            recipe_vectors.append(vector)
        recipe_names = genRecipeNames(recipes)
        print("Recipe Vectors", list(zip(recipe_names, recipe_vectors)))
        ing_vectors = list(zip(*recipe_vectors)) # Transpose (constraints are per-item, not per-recipe)
        target_vector = [targets.get(var, 0) for var in variables]
        
        return LpProject(inputs, explicit_inputs, outputs, targets, variables, recipe_names, ing_vectors, target_vector)
    
    @staticmethod
    def fromConfig(config_path):
        recipes = recipesFromConfig(config_path)
        return LpProject.fromRecipes(recipes)
    

def solveProblem(project):   
    
    additional_inputs = project.inputs -project.explicit_inputs
    print(additional_inputs)
    # Outputs that are only ever outputs (and not targets) are likely desired.        
    desired_byproducts = (project.outputs - project.inputs) - set(project.targets.keys())
    minimized_byproducts = set(project.variables) - desired_byproducts
    
    problem = LpProblem("gtnh_flow_lp_solver", LpMinimize)
    
    recipe_vars = LpVariable.dicts("recipe", project.recipe_names, 0)
    explicit_input_amounts = LpVariable.dicts("input", project.explicit_inputs, 0)
    additional_input_subtractors = LpVariable.dicts("in_sub", additional_inputs, 0, 1)
    additional_input_switches = LpVariable.dicts("in_switch", additional_inputs, 0, 1, cat=LpBinary)
    byproduct_amounts = LpVariable.dicts("byproduct", project.variables, 0)

    for item, recipe_coeffs, target in zip(project.variables, project.ing_matrix, project.target_vector):
        # To reach each target amount, use a combination of the recipe outputs and switched inputs
        # The amount for each input is unlimited, but there is a high cost for switching each one on
        if item in project.explicit_inputs:
            input_term = explicit_input_amounts[item]
        elif item in additional_inputs:
            # input_term = 0
            input_term = BIG_NUMBER * (additional_input_switches[item] - additional_input_subtractors[item])
            # Also restrict input to be strictly positive (no inputting negative items)
            problem += (additional_input_switches[item] - additional_input_subtractors[item]) >= 0 
        else:
            input_term = 0
        
        # Most of the time, products beyond the target should be heavily penalized
        # to encourage using other recipes to reprocess them.
        # However, some items are the output of byproduct processing (like uranium from radioactive waste byproduct)
        # our heuristic is that if something is only output (never an input), it should have a small production priority
        problem += lpDot(recipe_vars.values(), recipe_coeffs) + input_term == target + byproduct_amounts[item]
        # problem += lpDot(recipe_vars.values(), recipe_coeffs) + input_term == target

    desired_byproduct_amounts = [byproduct_amounts[var] for var in desired_byproducts]
    minimized_byproduct_amounts = [byproduct_amounts[var] for var in minimized_byproducts]
    print("Desired Byproducts:", desired_byproducts)
    print("Minimized Byproducts:", minimized_byproducts)
    problem += (priority_ratio**0 * lpSum(recipe_vars)
        + priority_ratio**1 * -1 * lpSum(desired_byproduct_amounts) # Byproducts are lowest priority (nearly)
        + priority_ratio**2 * lpSum(explicit_input_amounts)
        + priority_ratio**3 * -1 * lpSum(additional_input_subtractors) # invert (to max)
        + priority_ratio**3 * 2 * lpSum(additional_input_switches) # Subtractors are <= 1, so we can use smaller priority step
        + priority_ratio**3 * 4 * lpSum(minimized_byproduct_amounts) # Same as above
    )
    problem.solve()
    
    print("Problem Status:", LpStatus[problem.status], "in", problem.solutionCpuTime, "cpu-seconds")

    def show_values(var_dict):
        for var in var_dict:
            print(var, value(var_dict[var]))
            
    print("\nSwitches")
    show_values(additional_input_switches)
    print("\nSubtractors")
    show_values(additional_input_subtractors)
    print("\nRecipe vars")
    show_values(recipe_vars)
    print("\nExplicit Ingredient Vars")
    show_values(explicit_input_amounts)
    print("\nSurplus Vars")
    show_values(byproduct_amounts)

    print("\nConstraints: ")
    for item, recipe_coeffs, target in zip(project.variables, project.ing_matrix, project.target_vector):
        # To reach each target amount, use a combination of the recipe outputs and switched inputs
        # The amount for each input is unlimited, but there is a high cost for switching each one on
        if item in project.explicit_inputs:
            input_term = value(explicit_input_amounts[item])
        elif item in additional_inputs:
            input_term = BIG_NUMBER * (value(additional_input_switches[item]) - value(additional_input_subtractors[item]))
        else:
            input_term = 0
        recipe_sum = sum(value(v) * coeff for ((name, v), coeff) in zip(recipe_vars.items(), recipe_coeffs))
        coeff_string = " + ".join([f"{value(v)} * {coeff} [{name}]" for ((name, v), coeff) in zip(recipe_vars.items(), recipe_coeffs)])
        surplus_term = value(byproduct_amounts[item])
        print(item, f"{coeff_string} + {input_term} = {recipe_sum+input_term} == {target} + {surplus_term}")
    return problem
        
# recipes = recipesFromConfig("devtest/insufficient_byproduct_fake.yaml")
recipes = recipesFromConfig("devtest/bauxite.yaml")
# recipes = recipesFromConfig("devtest/palladium_loop.yaml")
problem = solveProblem(LpProject.fromRecipes(recipes))
            
    

{'calcite': 0, 'sodium hydroxide': 1, 'alumina': 2, 'stone dust': 3, 'purified bauxite': 4, 'quicklime': 5, 'iron dust': 6, 'steam': 7, 'copper dust': 8, 'silicon dioxide': 9, 'sluice juice': 10, 'oxygen': 11, 'carbon dust': 12, 'bauxite slag': 13, 'sodium carbonate': 14, 'aluminium hydroxide': 15, 'antimony dust': 16, 'aluminium dust': 17, 'water': 18, 'rutile': 19, 'sodium dust': 20, 'carbon dioxide': 21, 'nickel dust': 22, 'hydrogen': 23, 'tin dust': 24, 'gallium': 25, 'heated bauxite slurry': 26, 'bauxite slurry': 27, 'sodium aluminate': 28}
Recipe Vectors [('mixer_purified_bauxite', [0, -3, 0, 0, -32, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -5, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0]), ('oc_steam', [0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, -32, 0]), ('lcr_carbon_dioxide', [10, 0, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 16, 9, -1, 0, 0, 0, 0, 0, -5, 0, 0, 0, 0, -32, 0, 0]), ('lcr_calcite', [-5, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [26]:
problem.constraints

OrderedDict([('_C1', -1*in_sub_calcite + 1*in_switch_calcite + 0 >= 0),
             ('_C2',
              -1*byproduct_calcite + -1000.0*in_sub_calcite + 1000.0*in_switch_calcite + -5*recipe_lcr_calcite + 10*recipe_lcr_carbon_dioxide + 0.0 = 0),
             ('_C3',
              -1*in_sub_sodium_hydroxide + 1*in_switch_sodium_hydroxide + 0 >= 0),
             ('_C4',
              -1*byproduct_sodium_hydroxide + -1000.0*in_sub_sodium_hydroxide + 1000.0*in_switch_sodium_hydroxide + -16*recipe_lcr_aluminium_dust + 1*recipe_lcr_sodium_aluminate + 1*recipe_lcr_sodium_dust + -3*recipe_mixer_purified_bauxite + 0.0 = 0),
             ('_C5',
              -1*byproduct_alumina + 8*recipe_lcr_carbon_dioxide + -10 = 0),
             ('_C6',
              -1*byproduct_stone_dust + 1*recipe_centrifuge_sluice_juice + 0 = 0),
             ('_C7',
              -1*in_sub_purified_bauxite + 1*in_switch_purified_bauxite + 0 >= 0),
             ('_C8',
              -1*byproduct_purified_bauxite + -10

In [27]:
problem.variables

<bound method LpProblem.variables of gtnh_flow_lp_solver:
MINIMIZE
40000*byproduct_alumina + 40000*byproduct_aluminium_dust + 40000*byproduct_aluminium_hydroxide + -10*byproduct_antimony_dust + 40000*byproduct_bauxite_slag + 40000*byproduct_bauxite_slurry + 40000*byproduct_calcite + 40000*byproduct_carbon_dioxide + -10*byproduct_carbon_dust + -10*byproduct_copper_dust + -10*byproduct_gallium + 40000*byproduct_heated_bauxite_slurry + -10*byproduct_hydrogen + -10*byproduct_iron_dust + -10*byproduct_nickel_dust + -10*byproduct_oxygen + 40000*byproduct_purified_bauxite + 40000*byproduct_quicklime + -10*byproduct_rutile + -10*byproduct_silicon_dioxide + 40000*byproduct_sluice_juice + 40000*byproduct_sodium_aluminate + 40000*byproduct_sodium_carbonate + 40000*byproduct_sodium_dust + 40000*byproduct_sodium_hydroxide + 40000*byproduct_steam + -10*byproduct_stone_dust + -10*byproduct_tin_dust + 40000*byproduct_water + -1000*in_sub_aluminium_dust + -1000*in_sub_aluminium_hydroxide + -1000*in_sub

In [28]:
# That looks like it's working! Next on the chopping block is what I'm calling "Matrix Normalization" 
# ... although I'm not even sure that's the right name.
# The idea is to scale both item and recipe vectors so that the largest value in each is 1, 
# ... AND we minimize the ratio between the largest and smallest values in the matrix (which we use as the priority ratio)
# This should make the LP solver more numerically stable and less likely to run into precision issues.
# It will also help with the "1 trillion or so ought to do" problem, as we can use a smaller BIG_NUMBER,
# and help put items in the same priority level on the same-ish scale.

# The rows can be scaled for free - divide the row (all ingredients) by the largest value in the row.
# The challenge is finding a scale factor for the *columns* which minimizes the ratio between the largest and smallest
# ... values in the *matrix*.

# One way to formulate this in LP would be to minimize the difference between
# ... the largest and smallest values in the values, which are themselves variables that are solved for.

# I'mma ask ChatGPT for this one... oh boy did GPT deliver. I love this stuff.

# Here's its write-up:
# This approach aims to scale a matrix such that each column is adjusted by a 
# scaling factor to ensure numerical stability in linear programming (LP) solvers. 
# First, each vector (row) in the matrix is normalized so that the largest value in 
# each vector becomes 1. Using the PuLP library, an optimization problem is then set 
# up with the objective of minimizing the difference between the maximum and minimum 
# scaled values in the matrix. The variables in this optimization problem are the 
# scaling factors for each column. Constraints are added to ensure that each element 
# in the matrix, when scaled, lies between the defined maximum and minimum values. 
# The optimization problem is solved to determine the optimal scaling factors, 
# which are then applied to the normalized matrix. 
# This process ensures the matrix values are balanced and the ratio between the 
# largest and smallest values is minimized, enhancing the numerical stability of 
# LP solvers.


In [30]:
# ... and here's the routine:
import pulp

# Function to normalize a matrix with the added constraint
def pulp_scale_matrix(matrix):
    
    num_rows = len(matrix)
    num_cols = len(matrix[0])
   
    # Create a PuLP problem instance
    prob = pulp.LpProblem("MinimizeMatrixRatio", pulp.LpMinimize)
    
    # Variables: scaling factors for each row and column
    row_scale_factors = [pulp.LpVariable(f"rs_{i}", lowBound=1) for i in range(num_rows)]
    
    # Constraints and objective function
    max_val = pulp.LpVariable("max_val", lowBound=0)
    min_val = pulp.LpVariable("min_val", lowBound=0)
    
    # Define constraints and objective for each element in the matrix
    for i in range(num_rows):
        for j in range(num_cols):
            if abs(matrix[i][j]) > 0:
                scaled_value = abs(matrix[i][j]) * row_scale_factors[i]
                prob += max_val >= scaled_value
                prob += min_val <= scaled_value
    
    # Objective: min (max_val - min_val)
    prob += max_val - min_val
    
    # Solve the problem
    prob.solve()
    
    # Extract the scale factors
    row_scale_factors = [scale_factor.varValue for scale_factor in row_scale_factors]
    
    # Apply the scaling factors to the normalized matrix
    scaled_matrix = [[matrix[i][j] * row_scale_factors[i] for j in range(num_cols)] for i in range(num_rows)]
    
    return row_scale_factors, scaled_matrix
    # print(value(min_val), value(max_val))
    # return row_scales, col_scales, scaled_matrix

# Example usage
pall_loop = [
    [-1000, 0,      0],
    [1000,  -9000,  0],
    [-1,    -9,     0.95],
    [0,     16,     -1],
    [0,     2,      0],
]

pulp_row_scales, pulp_matrix = pulp_scale_matrix(pall_loop)
for row in pulp_matrix:
    print(row)

[-9000.0, 0.0, 0.0]
[1000.0, -9000.0, 0.0]
[-592.10526, -5328.947340000001, 562.499997]
[0.0, 9000.0, -562.5]
[0.0, 9000.0, 0.0]


In [31]:
# I think solving jointly for both row and column scale factors will yield a better ratio, but that requires
# some quadratic solving. I think the cvxpy library may do the trick, but I want to confirm that the 
# ratio improvement is actually good enough for adding the dependency (or switching to it from pulp)

import cvxpy as cp
import numpy as np

def qp_scale_matrix(matrix, target_vector):
    # Augment the matrix with the target vector
    aug_matrix = np.hstack([matrix, np.array(target_vector).reshape((-1,1))])
    
    num_rows = len(aug_matrix)
    num_cols = len(aug_matrix[0])
    
    # Variables: scaling factors for each row and column
    item_scales_vars = cp.Variable(num_rows, pos=True)
    recipe_scales_vars = cp.Variable(num_cols, pos=True)
    
    # Constraints and objective function
    constraints = []
    max_val = cp.Variable(pos=True)
    min_val = cp.Variable(pos=True)
    
    # Have the solver normalize to max_val of 1
    # constraints.append(max_val == 1)
    # Actually, the way I set up prioritization requires values >=1, so le's normalize that way
    constraints.append(min_val == 1)
    
    # Define constraints and objective for each element in the matrix
    for i in range(num_rows):
        for j in range(num_cols):
            if abs(aug_matrix[i][j]) > 0:
                scaled_value = abs(aug_matrix[i][j]) * item_scales_vars[i] * recipe_scales_vars[j]
                # scaled_value = abs(matrix[i][j]) * item_scales[i]
                constraints.append(max_val >= scaled_value)
                constraints.append(min_val <= scaled_value)
    
    objective = cp.Minimize(max_val / min_val)
    
    # Create the problem
    problem = cp.Problem(objective, constraints)
    
    # Solve the problem
    problem.solve(gp=True)
    print("Solve time", problem.solver_stats.solve_time)
    
    # Extract the scale factors
    item_scales = item_scales_vars.value
    recipe_scales = recipe_scales_vars.value[:-1]
    target_scale = recipe_scales_vars.value[-1]
    
    
    # Apply the scaling factors to the normalized matrix
    # recipe_scales = [2,1,1]
    scaled_aug_matrix = np.multiply(aug_matrix, np.outer(item_scales, recipe_scales_vars.value))
    # scaled_matrix = np.diag(item_scales) @ matrix
    
    # Un-augment the matrix
    scaled_matrix = scaled_aug_matrix[:,:-1]
    scaled_target_vector = scaled_aug_matrix[:, -1]
    
    return item_scales, recipe_scales, target_scale, scaled_matrix, scaled_target_vector


def ratio_solve_test(matrix, target):
    pulp_row_scales, pulp_matrix = pulp_scale_matrix(matrix)
    pulp_matrix = np.array(pulp_matrix)
    qp_row_scales, qp_col_scales, qp_target_scale, qp_matrix, qp_target = qp_scale_matrix(matrix, target)
    og_nonzero_abs = np.abs(matrix[np.nonzero(matrix)])
    pulp_nonzero_abs = np.abs(pulp_matrix[np.nonzero(pulp_matrix)])
    qp_nonzero_abs = np.abs(qp_matrix[np.nonzero(np.array(qp_matrix))])
    print("Original Ratio", np.max(og_nonzero_abs) / np.min(og_nonzero_abs))
    print("LP Ratio", np.max(pulp_nonzero_abs) / np.min(pulp_nonzero_abs))
    print("QP Ratio", np.max(qp_nonzero_abs) / np.min(qp_nonzero_abs))
    return qp_matrix
display(ratio_solve_test(np.array(pall_loop), np.array([0,0,0,0,25])))
display(np.array(pall_loop))
# That is dramatic... but I want to make sure I haven't optimized too close to the sun.
# After doing this, does the LP solution (after getting un-scaled) still work?

Solve time 0.0001523
Original Ratio 9473.684210526317
LP Ratio 16.000000085333333
QP Ratio 1.2995725793787345


array([[-1.13998797,  0.        ,  0.        ],
       [ 1.19087732, -1.09127326,  0.        ],
       [-1.09127326, -1.        ,  1.29957258],
       [ 0.        ,  1.29957258, -1.        ],
       [ 0.        ,  1.13998797,  0.        ]])

array([[-1.0e+03,  0.0e+00,  0.0e+00],
       [ 1.0e+03, -9.0e+03,  0.0e+00],
       [-1.0e+00, -9.0e+00,  9.5e-01],
       [ 0.0e+00,  1.6e+01, -1.0e+00],
       [ 0.0e+00,  2.0e+00,  0.0e+00]])

In [32]:
project = LpProject.fromConfig("devtest/palladium_loop.yaml")
unscaled_sol = solveProblem(project)
item_scales, recipe_scales, target_scale, scaled_matrix, scaled_target = qp_scale_matrix(np.array(project.ing_matrix), project.target_vector)
project.ing_matrix = scaled_matrix.tolist()
project.target_vector = scaled_target
scaled_sol = solveProblem(project)
scaled_matrix

{'palladium salt dust': 0, 'reprecipitated palladium dust': 1, 'palladium enriched ammonia': 2, 'palladium metallic powder dust': 3, 'ammonia': 4}
Recipe Vectors [('lcr_palladium_metallic_powder', [0, 0, 1, -1, -1]), ('lcr_palladium_enriched_ammoni', [16, 2, -9, -9, 0]), ('is_palladium_salt_dust', [-1, 0, 0, 0.95, 0])]
{'palladium salt dust', 'palladium metallic powder dust', 'ammonia'}
Desired Byproducts: set()
Minimized Byproducts: {'palladium enriched ammonia', 'palladium salt dust', 'palladium metallic powder dust', 'ammonia', 'reprecipitated palladium dust'}
Problem Status: Optimal in 0.046000000002095476 cpu-seconds

Switches
palladium salt dust 0.0
palladium metallic powder dust 0.0
ammonia 1.0

Subtractors
palladium salt dust 0.0
palladium metallic powder dust 0.0
ammonia 0.969

Recipe vars
lcr_palladium_metallic_powder 31.0
lcr_palladium_enriched_ammoni 5.0
is_palladium_salt_dust 80.0

Explicit Ingredient Vars
palladium enriched ammonia 14.0

Surplus Vars
palladium salt dust 0

array([[ 0.        ,  1.29957258, -1.        ],
       [ 0.        ,  1.13998797,  0.        ],
       [ 1.19087732, -1.09127326,  0.        ],
       [-1.09127326, -1.        ,  1.29957258],
       [-1.13998797,  0.        ,  0.        ]])

In [33]:
print(unscaled_sol.variablesDict()["recipe_lcr_palladium_metallic_powder"].varValue,
scaled_sol.variablesDict()["recipe_lcr_palladium_metallic_powder"].varValue * recipe_scales[0]/target_scale)

31.0 30.99999999396295


In [34]:
# It should be possible to unscale and recover the original matrix
np.diag(1/item_scales) @ scaled_matrix @ np.diag(1/np.array(recipe_scales))
# ... Yep

array([[ 0.  , 16.  , -1.  ],
       [ 0.  ,  2.  ,  0.  ],
       [ 1.  , -9.  ,  0.  ],
       [-1.  , -9.  ,  0.95],
       [-1.  ,  0.  ,  0.  ]])

In [46]:
# Always fun to see math work. Lets test for ratio performance on other projects.
def descale_recipe_vars(recipe_vars, recipe_scales, target_scale):
    return np.array(recipe_vars) * recipe_scales / target_scale

def ratio_solve_test(project:LpProject):
    og_matrix = np.array(project.ing_matrix)
    pulp_row_scales, pulp_matrix = pulp_scale_matrix(project.ing_matrix)
    pulp_matrix = np.array(pulp_matrix)
    qp_row_scales, qp_col_scales, qp_target_scale, qp_matrix, qp_target = qp_scale_matrix(project.ing_matrix, project.target_vector)
    og_nonzero_abs = np.abs(og_matrix[np.nonzero(og_matrix)])
    pulp_nonzero_abs = np.abs(pulp_matrix[np.nonzero(pulp_matrix)])
    qp_nonzero_abs = np.abs(qp_matrix[np.nonzero(np.array(qp_matrix))])
    
    print("Original Ratio", np.max(og_nonzero_abs) / np.min(og_nonzero_abs))
    print("LP Ratio", np.max(pulp_nonzero_abs) / np.min(pulp_nonzero_abs))
    print("QP Ratio", np.max(qp_nonzero_abs) / np.min(qp_nonzero_abs))
    
    unscaled_sol = solveProblem(project)
    
    project.ing_matrix = qp_matrix.tolist()
    project.target_vector = qp_target
    scaled_sol = solveProblem(project)
    
    recipe_vars = [var_name for var_name in unscaled_sol.variablesDict().keys() if var_name.startswith("recipe_")]
    unscaled_vals = [value(unscaled_sol.variablesDict()[var_name]) for var_name in recipe_vars]
    scaled_vals = [value(scaled_sol.variablesDict()[var_name]) for var_name in recipe_vars]
    descaled_vals = descale_recipe_vars(scaled_vals, qp_col_scales, qp_target_scale)
    
    np.testing.assert_almost_equal(unscaled_vals, descaled_vals, decimal=4)
    
    return qp_matrix

ratio_solve_test(LpProject.fromConfig("devtest/insufficient_byproduct_fake.yaml"))

{'hydrogen': 0, 'calcium chloride dust': 1, 'sodium hydroxide dust': 2, 'tungsten dust': 3, 'water': 4, 'scheelite dust': 5, 'salt': 6, 'sodium tungstate': 7}
Recipe Vectors [('lcr_sodium_tungstate', [0, -6, 0, 0, 0, 7, 8, -4]), ('cr_salt', [0.5, 0, 1, 0, -1, 0, -2, 0]), ('ebf_scheelite_dust', [-3, 0, 0, 1, 0, -7, 0, 0])]
Solve time 0.0001542
Original Ratio 16.0
LP Ratio 6.0
QP Ratio 1.144714242553332
{'scheelite dust', 'salt', 'hydrogen', 'water'}
Desired Byproducts: {'sodium hydroxide dust'}
Minimized Byproducts: {'hydrogen', 'calcium chloride dust', 'tungsten dust', 'water', 'scheelite dust', 'salt', 'sodium tungstate'}
Problem Status: Optimal in 0.07799999999406282 cpu-seconds

Switches
scheelite dust 1.0
salt 0.0
hydrogen 1.0
water 0.0

Subtractors
scheelite dust 0.9986
salt 0.0
hydrogen 0.9994
water 0.0

Recipe vars
lcr_sodium_tungstate 0.0
cr_salt 0.0
ebf_scheelite_dust 0.2

Explicit Ingredient Vars
calcium chloride dust 0.0
sodium tungstate 0.0

Surplus Vars
hydrogen 0.0
calciu

array([[ 0.        ,  1.        , -1.14471424],
       [-1.06991319,  0.        ,  0.        ],
       [ 0.        ,  1.06991319,  0.        ],
       [ 0.        ,  0.        ,  1.06991319],
       [ 0.        , -1.06991319,  0.        ],
       [ 1.14471424,  0.        , -1.        ],
       [ 1.        , -1.14471424,  0.        ],
       [-1.06991319,  0.        ,  0.        ]])

In [47]:
ratio_solve_test(LpProject.fromConfig("devtest/bauxite.yaml"))

{'calcite': 0, 'sodium hydroxide': 1, 'alumina': 2, 'stone dust': 3, 'purified bauxite': 4, 'quicklime': 5, 'iron dust': 6, 'steam': 7, 'copper dust': 8, 'silicon dioxide': 9, 'sluice juice': 10, 'oxygen': 11, 'carbon dust': 12, 'bauxite slag': 13, 'sodium carbonate': 14, 'aluminium hydroxide': 15, 'antimony dust': 16, 'aluminium dust': 17, 'water': 18, 'rutile': 19, 'sodium dust': 20, 'carbon dioxide': 21, 'nickel dust': 22, 'hydrogen': 23, 'tin dust': 24, 'gallium': 25, 'heated bauxite slurry': 26, 'bauxite slurry': 27, 'sodium aluminate': 28}
Recipe Vectors [('mixer_purified_bauxite', [0, -3, 0, 0, -32, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -5, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0]), ('oc_steam', [0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, -32, 0]), ('lcr_carbon_dioxide', [10, 0, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 16, 9, -1, 0, 0, 0, 0, 0, -5, 0, 0, 0, 0, -32, 0, 0]), ('lcr_calcite', [-5, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

array([[ 0.        ,  0.        ,  1.74813721, -2.28814992,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-3.41563734,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  3.08360799, -1.94129208,  1.06626983],
       [ 0.        ,  0.        ,  1.99999995,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         2.00000002,  0.        ,  0.        ,  0.        ,  0.        ],
       [-2.00000006,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.05498533,  0.        ,  0.        ,  2.50985128,  3.83503228,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  4.        ,
         1.        ,  0.        ,  0.        

In [48]:
ratio_solve_test(LpProject.fromConfig("devtest/baux_to_titanium.yaml"))

{'aluminium dust': 0, 'bauxite dust': 1, 'chlorine': 2, 'rutile dust': 3, 'salt': 4, 'sodium dust': 5, 'hot titanium ingot': 6, 'titanium tetrachloride': 7, 'carbon monoxide': 8, 'magnesium dust': 9, 'hydrogen': 10, 'titanium ingot': 11, 'oxygen': 12, 'carbon dust': 13, 'magnesium chloride dust': 14}
Recipe Vectors [('electrolyzer_bauxite_dust', [16, -39, 0, 2, 0, 0, 0, 0, 0, 0, 10, 0, 11, 0, 0]), ('lcr_rutile_dust', [0, 0, -4, -1, 0, 0, 0, 1, 2, 0, 0, 0, 0, -2, 0]), ('ebf_magnesium_dust', [0, 0, 0, 0, 0, 0, 1, -1, 0, -2, 0, 0, 0, 0, 6]), ('vf_hot_titanium_ingot', [0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0]), ('lcr_magnesium_chloride_dust', [0, 0, 0, 0, 8, -4, 0, 0, 0, 2, 0, 0, 0, 0, -6]), ('electrolyzer_salt', [0, 0, 1, 0, -2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('electrolyzer_carbon_monoxide', [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 1, 0])]
Solve time 0.0002906
Original Ratio 39.0
LP Ratio 11.000000141025643
QP Ratio 1.4010196657738068
{'titanium tetrachloride', 'magnesium dust', 'ca

array([[ 1.18364676,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-1.18364676,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -1.18364677,  0.        ,  0.        ,  0.        ,
         1.18364676,  0.        ],
       [ 1.        , -1.40101966,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.18364676,
        -1.18364677,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.18364676,
         1.18364677,  0.        ],
       [ 0.        ,  0.        ,  1.18364677, -1.18364676,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.18364677, -1.18364676,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        , -1.40101967],
       [ 0.        ,  0.    

In [49]:
ratio_solve_test(LpProject.fromConfig("devtest/germanium.yaml"))

{'small pile of iron dust': 0, 'lead-silica residue dust': 1, 'sodium hydroxide': 2, 'hydrochloric acid': 3, 'germanium oxalate solution': 4, 'zinc leach': 5, 'iron dust': 6, 'natural gas': 7, 'quicklime dust': 8, 'steam': 9, 'zinc leach residue dust': 10, 'waelz slag dust': 11, 'germanium-gallium extract': 12, 'ultra pure water': 13, 'carbon monoxide': 14, 'germanium dust': 15, 'germanium tetrachloride': 16, 'distilled water': 17, 'oxygen': 18, 'carbon dust': 19, 'sulfuric acid': 20, 'germanium rich oxide dust': 21, 'oxalic acid solution': 22, 'germanium leach': 23, 'germanium dioxide dust': 24, 'impure germanium tetrachloride': 25, 'waelz oxide dust': 26, 'germanium concentrate dust': 27, 'germanium extract': 28, 'chlorine': 29, 'hematite dust': 30, 'wood log': 31, 'water': 32, 'purified germanium oxalate leach': 33, 'diluted sulfuric acid': 34, 'iron oxalate dihydrate dust': 35, 'oxalic acid': 36, 'carbon dioxide': 37, 'anglesite dust': 38, 'germanium-gallium extraction mixture': 39

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.64754898,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [64]:
ratio_solve_test(LpProject.fromConfig("223_monazite_line.yaml"))
# I expected that one to fail. With an original priority ratio of 360000, 
# the solver wouldn't be able to solve this problem without normalization,
# because 360000**4 = far too large for double precision.
# However, it is very cool (and unexpected) for the normalizer to solve
# the ratio for this complex problem down to just 2.0

{'acidic monazite powder dust': 0, 'water1': 1, 'ammonium nitrate solution': 2, 'neutralized monazite rare earth filtrate dust': 3, 'samaric residiue dust': 4, 'tiny pile of hafnia-zirconia blend dust': 5, 'chloromethane': 6, 'nitric acid': 7, 'hydrofluoric acid': 8, 'oxalate': 9, 'sulfur': 10, 'thorium-phosphate cake dust': 11, 'uranium-238': 12, 'thorium-phosphate concentrate dust': 13, 'ammonium chloride': 14, 'oxygen': 15, 'iodine': 16, 'cerium III oxide dust': 17, 'uranium-235': 18, 'hafnium runoff dust': 19, 'diluted monazite sulfate': 20, 'magnesium': 21, 'cerium dioxide dust': 22, 'saltpeter': 23, 'heterogenous halogenic monazite rare earth mixture dust': 24, 'zirconium tetrachloride dust': 25, 'diluted monazite rare earth mud': 26, 'hafnium ingot': 27, 'crushed monazite ore': 28, 'chlorine': 29, 'water cell': 30, 'salt': 31, 'low-purity hafnium dust': 32, 'muddy monazite rare earth solution': 33, 'uraniumfiltrate dust': 34, 'hafnium tetrachloride solution': 35, 'gadolinium dus

AssertionError: 
Arrays are not almost equal to 4 decimals

Mismatched elements: 3 / 37 (8.11%)
Max absolute difference: 1000.
Max relative difference: 3.96780207e-09
 x: array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,...
 y: array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

In [70]:
ratio_solve_test(LpProject.fromConfig("228_platline.yaml"))

{'iridium dust': 0, 'potassium disulfate dust': 1, 'aqua regia': 2, 'rhodium salt dust': 3, 'nitric acid': 4, 'hot ruthenium tetroxide solution': 5, 'rarest metal residue dust': 6, 'copper dust': 7, 'platinum salt dust': 8, 'ethylene': 9, 'carbon monoxide': 10, 'ammonium chloride': 11, 'calcium chloride dust': 12, 'oxygen': 13, 'carbon dust': 14, 'refined platinum salt dust': 15, 'iridium dioxide dust': 16, 'PMP': 17, 'ruthenium tetroxide solution': 18, 'saltpeter': 19, 'zinc sulfate dust': 20, 'acidic iridium solution': 21, 'reprecipitated platinum dust': 22, 'rhodium filter cake dust': 23, 'sludge dust residue dust': 24, 'chlorine': 25, 'rhodium nitrate': 26, 'salt': 27, 'diluted sulfuric acid': 28, 'sodium dust': 29, 'ruthenium tetroxide dust': 30, 'rhodium salt solution': 31, 'rhodium filter cake solution': 32, 'iridium metal residue dust': 33, 'osmium solution': 34, 'calcium dust': 35, 'palladium enriched ammonia': 36, 'sulfur dust': 37, 'ruthenium tetroxide': 38, 'ammonia': 39, '

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-3.38414261,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])